In [1]:
from warnings import onceregistry

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import FeatureHasher

In [2]:
df_wines = pd.read_csv("XWines_Slim_1K_wines.csv")
df_ratings = pd.read_csv("XWines_Slim_150K_ratings.csv")

C:\Users\vince\AppData\Local\Temp\ipykernel_27104\174622797.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ratings = pd.read_csv("XWines_Slim_150K_ratings.csv")


In [3]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,Do Lugar Moscatel Espumantes,Sparkling,Varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10012,Dal Pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [4]:
df_ratings.head()

,RatingID,UserID,WineID,Vintage,Rating,Date
0,143,1356810,103471,1950,4.5,2021-11-02 20:52:59
1,199,1173759,111415,1951,5.0,2015-08-20 17:46:26
2,348,1164877,111395,1952,5.0,2020-11-13 05:40:26
3,374,1207665,111433,1953,5.0,2017-05-05 06:44:13
4,834,1075841,111431,1955,5.0,2016-09-14 20:18:38


In [5]:
wine_ids = df_wines['WineID'].copy()

In [6]:
preprocessing = ['WineName', 'Type', 'Elaborate', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
for col in preprocessing:
    df_wines[col] = df_wines[col].str.lower()

In [7]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,espumante moscatel,sparkling,varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,ancellotta,red,varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,medium-bodied,medium,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,cabernet sauvignon,red,varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,full-bodied,high,BR,brazil,1001,serra gaúcha,10002,castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,maison de ville cabernet-merlot,red,assemblage/bordeaux red blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,full-bodied,medium,BR,brazil,1001,serra gaúcha,10000,aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,do lugar moscatel espumantes,sparkling,varietal/100%,['Muscat/Moscato Bianco'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10012,dal pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [8]:
preprocessing2 = ['Grapes', 'Harmonize']

for col in preprocessing2:
    if col in df_wines.columns:
        # 1. Fill NaN with empty string FIRST, then ensure string type
        #    This prevents np.nan from becoming the string 'nan'
        series = df_wines[col].fillna('').astype(str) # <-- Swapped order here

        # 2. Remove unwanted characters
        series = series.str.replace('[', '', regex=False)
        series = series.str.replace(']', '', regex=False)
        series = series.str.replace("'", '', regex=False)
        series = series.str.replace('/', '', regex=False) # Remove slash

        # 3. Split into a Series of lists
        series_list = series.str.split(',')

        # 4. Apply cleaning (strip) AND lowercasing to items WITHIN each list
        cleaned_series_list = series_list.apply(
            lambda lst: [item.strip().lower() for item in lst if isinstance(item, str)] # Apply strip() and lower()
            if isinstance(lst, list) else # Check if the element is actually a list
            lst # Handle potential non-list elements
        )

        # 5. Assign back, converting potential [''] resulting from empty strings OR original NaNs to []
        df_wines[col] = cleaned_series_list.apply(lambda x: [] if x == [''] else x)

    else:
         print(f"Warning: Column '{col}' not found in DataFrame.")

In [9]:
df_wines.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,espumante moscatel,sparkling,varietal/100%,[muscatmoscato],"[pork, rich fish, shellfish]",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,ancellotta,red,varietal/100%,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",12.0,medium-bodied,medium,BR,brazil,1001,serra gaúcha,10001,casa perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,cabernet sauvignon,red,varietal/100%,[cabernet sauvignon],"[beef, lamb, poultry]",12.0,full-bodied,high,BR,brazil,1001,serra gaúcha,10002,castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100005,maison de ville cabernet-merlot,red,assemblage/bordeaux red blend,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",11.0,full-bodied,medium,BR,brazil,1001,serra gaúcha,10000,aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100007,do lugar moscatel espumantes,sparkling,varietal/100%,[muscatmoscato bianco],"[pork, rich fish, shellfish]",7.5,medium-bodied,high,BR,brazil,1001,serra gaúcha,10012,dal pizzol,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."


In [10]:
df_wines['ABV'] = df_wines['ABV'].astype(float)

In [11]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WineID      1007 non-null   int64  
 1   WineName    1007 non-null   object 
 2   Type        1007 non-null   object 
 3   Elaborate   1007 non-null   object 
 4   Grapes      1007 non-null   object 
 5   Harmonize   1007 non-null   object 
 6   ABV         1007 non-null   float64
 7   Body        1007 non-null   object 
 8   Acidity     1007 non-null   object 
 9   Code        1007 non-null   object 
 10  Country     1007 non-null   object 
 11  RegionID    1007 non-null   int64  
 12  RegionName  1007 non-null   object 
 13  WineryID    1007 non-null   int64  
 14  WineryName  1007 non-null   object 
 15  Website     900 non-null    object 
 16  Vintages    1007 non-null   object 
dtypes: float64(1), int64(3), object(13)
memory usage: 133.9+ KB


In [12]:
categorical_features = ['Type', 'Elaborate', 'Body', 'Acidity', 'Country']
df_wines = pd.get_dummies(df_wines, columns=categorical_features)

In [13]:
df_wines.head()

,WineID,WineName,Grapes,Harmonize,ABV,Code,RegionID,RegionName,WineryID,WineryName,...,Country_russia,Country_slovenia,Country_south africa,Country_spain,Country_switzerland,Country_turkey,Country_ukraine,Country_united kingdom,Country_united states,Country_uruguay
0,100001,espumante moscatel,[muscatmoscato],"[pork, rich fish, shellfish]",7.5,BR,1001,serra gaúcha,10001,casa perini,...,False,False,False,False,False,False,False,False,False,False
1,100002,ancellotta,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",12.0,BR,1001,serra gaúcha,10001,casa perini,...,False,False,False,False,False,False,False,False,False,False
2,100003,cabernet sauvignon,[cabernet sauvignon],"[beef, lamb, poultry]",12.0,BR,1001,serra gaúcha,10002,castellamare,...,False,False,False,False,False,False,False,False,False,False
3,100005,maison de ville cabernet-merlot,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",11.0,BR,1001,serra gaúcha,10000,aurora,...,False,False,False,False,False,False,False,False,False,False
4,100007,do lugar moscatel espumantes,[muscatmoscato bianco],"[pork, rich fish, shellfish]",7.5,BR,1001,serra gaúcha,10012,dal pizzol,...,False,False,False,False,False,False,False,False,False,False


In [14]:
grapes_text = df_wines['Grapes'].apply(lambda lst: ' '.join(lst))
harmonize_text = df_wines['Harmonize'].apply(lambda lst: ' '.join(lst))
corpus = grapes_text + ' ' + harmonize_text

vect = TfidfVectorizer()
tfidf_matrix = vect.fit_transform(corpus)


In [15]:
scaler = StandardScaler()
df_wines['ABV'] = scaler.fit_transform(df_wines[['ABV']])

In [16]:
df_wines['RegionName'].value_counts()

RegionName
serra gaúcha            32
mendoza                 32
central valley (cl)     29
napa valley             27
douro                   22
                        ..
carignano del sulcis     1
bardolino classico       1
fiano di avellino        1
collio                   1
südsteiermark            1
Name: count, Length: 324, dtype: int64

In [17]:
N_FEATURES_HASH = 16
hasher = FeatureHasher(n_features=N_FEATURES_HASH, input_type='string', alternate_sign=False)
region_feature_values = [[region] for region in df_wines['RegionName']]
hashed_features_sparse = hasher.fit_transform(region_feature_values)
hashed_feature_columns = [f'RegionHash_{i}' for i in range(N_FEATURES_HASH)]
df_hashed_regions = pd.DataFrame(hashed_features_sparse.toarray(),
columns=hashed_feature_columns,
index=df_wines.index)


In [18]:
df_wines = pd.concat([df_wines.drop(['RegionName'], axis=1), df_hashed_regions], axis=1)

In [19]:
df_wines = df_wines.drop(['WineryName','WineName'], axis=1)

In [20]:
df_wines.head()

,WineID,Grapes,Harmonize,ABV,Code,RegionID,WineryID,Website,Vintages,Type_dessert,...,RegionHash_6,RegionHash_7,RegionHash_8,RegionHash_9,RegionHash_10,RegionHash_11,RegionHash_12,RegionHash_13,RegionHash_14,RegionHash_15
0,100001,[muscatmoscato],"[pork, rich fish, shellfish]",-1.774251,BR,1001,10001,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100002,[ancellotta],"[beef, barbecue, codfish, pasta, pizza, cheese]",-0.408248,BR,1001,10001,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100003,[cabernet sauvignon],"[beef, lamb, poultry]",-0.408248,BR,1001,10002,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100005,"[cabernet sauvignon, merlot]","[beef, lamb, game meat, poultry]",-0.711804,BR,1001,10000,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,[muscatmoscato bianco],"[pork, rich fish, shellfish]",-1.774251,BR,1001,10012,http://www.dalpizzol.com.br,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201...",False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
from scipy.sparse import csr_matrix

non_text_columns = ['ABV'] + [col for col in df_wines.columns if col.startswith('type_')
or col.startswith('elaborate_')
or col.startswith('body_')
or col.startswith('acidity_')
or col.startswith('country_')]
non_text_columns = [col for col in non_text_columns if col not in hashed_feature_columns]

if non_text_columns:
    numeric_sparse = csr_matrix(df_wines[non_text_columns].values)
else:
    numeric_sparse = None


In [24]:
from scipy.sparse import hstack

matrices = []
if numeric_sparse is not None:
    matrices.append(numeric_sparse)
    matrices.append(tfidf_matrix)
    matrices.append(hashed_features_sparse)

combined_sparse = hstack(matrices)
print("Combined feature matrix shape:", combined_sparse.shape)

Combined feature matrix shape: (1007, 278)


In [26]:
combined_dense = combined_sparse.toarray()
embedding_df = pd.DataFrame(combined_dense, index=wine_ids)
embedding_df.index.name = "WineID"

In [27]:
embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,268,269,270,271,272,273,274,275,276,277
WineID,,,,,,,,,,,,,,,,,,,,,
100001,-1.774251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,-0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,-0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,-0.711804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100007,-1.774251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(combined_sparse)

In [29]:
similarity_df = pd.DataFrame(similarity_matrix, index=wine_ids, columns=wine_ids)

In [30]:
similarity_df.head()

WineID,100001,100002,100003,100005,100007,100008,100010,100012,100013,100014,...,198580,198885,199204,199306,199360,199408,199481,199533,199885,200139
WineID,,,,,,,,,,,,,,,,,,,,,
100001,1.000000,0.516307,0.516307,0.629946,0.952912,0.285456,-0.108890,0.545759,-0.623514,-0.623514,...,0.081255,0.241784,0.601641,0.565256,0.019903,0.012275,-0.108890,0.317677,0.231239,0.057731
100002,0.516307,1.000000,0.555854,0.565764,0.516307,0.488982,-0.027780,0.189163,-0.208467,-0.208467,...,0.020476,0.094202,0.543265,0.176538,-0.009229,0.038551,-0.030144,0.066188,0.066188,0.070012
100003,0.516307,0.555854,1.000000,0.871542,0.516307,0.951008,0.072776,0.189163,-0.213585,-0.213585,...,0.020476,0.101021,0.573418,0.196291,0.013615,-0.009229,0.024082,0.104534,0.088385,0.091915
100005,0.629946,0.565764,0.871542,1.000000,0.629946,0.761955,0.014089,0.306634,-0.353273,-0.353273,...,0.033191,0.141283,0.613129,0.299768,0.021178,0.001845,0.031678,0.133691,0.142405,0.082375
100007,0.952912,0.516307,0.516307,0.629946,1.000000,0.285456,-0.108890,0.542749,-0.623514,-0.623514,...,0.075553,0.235748,0.587169,0.548891,0.008771,0.002992,-0.108890,0.285907,0.220422,0.057731
